# Assignment 1 - INF367

In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.visualization import array_to_latex, plot_histogram
import numpy as np
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector, Operator


# To be used for measurments

backend = QasmSimulator()
def measure(qc: QuantumCircuit, shots=1000):
    qc.measure_all()
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc, shots=shots)
    result = job.result()
    return result.get_counts()

### 1.1 Playing with Quantum Circuits

1. Implement circuit to produce  |ψ⟩ = 1√2 ( |RRLL⟩ − |LLRR⟩ )

In [ ]:
qc = QuantumCircuit(4)

qc.h(3)
qc.h(2)
qc.h(1)
qc.h(0)

qc.u(np.pi*2.5, np.pi*2.5, np.pi*2,1)

qc.cx(0,1)
qc.cx(2,1)
qc.cx(3,2)

statevector = Statevector(qc)
x = Operator(qc).to_matrix()

display(statevector.draw('latex'))
display(qc.draw(reverse_bits=True))
display(array_to_latex(x))


2. Full standard measurment

In [3]:
res = measure(qc, shots=1000)


In [ ]:
print('Circuit:')
display(qc.draw(reverse_bits=True))

print('Result:',res,end='\n\n')

print('Histogram:')
plot_histogram(res)

3. Inverese circuit

In [ ]:
# Original circuit

qc = QuantumCircuit(4)

qc.h(3)
qc.h(2)
qc.h(1)
qc.h(0)

qc.u(np.pi*2.5,    np.pi*2.5,   np.pi*2,   1)

qc.cx(0,1)
qc.cx(2,1)
qc.cx(3,2)

qc.barrier()

# Inversing the circuit

qc.cx(3,2)
qc.cx(2,1)
qc.cx(0,1)

qc.u(-1*np.pi*2.5, -1*np.pi*2, -1*np.pi*2.5, 1)
 
qc.barrier()
qc.h(3)
qc.h(2)
qc.h(1)
qc.h(0)

statevector = Statevector(qc)
x_inverse = Operator(qc).to_matrix()

print('Statevector:')
display(statevector.draw('latex'))

print('Circuit:')
display(qc.draw(reverse_bits=True, plot_barriers=False))

print('Output matrix:')
display(array_to_latex(x_inverse))


We flipped the order of the gates, and found that the U-gate had to be inversed. We looked at the conjugated and transformed version of the matrix concluding that $\phi$ and $\lambda$ had to be negative and change place. Due to the negativity of sin when flipping the matrix, $\theta$ had to be negative. Having a negative $\theta$ would not affect the value of cos.

We also found that qiskit had a method for inversing a circuit, and we used this to doublecheck our result.

In [ ]:
qc = QuantumCircuit(4)

qc.h(3)
qc.h(2)
qc.h(1)
qc.h(0)

qc.u(np.pi*2.5,    np.pi*2.5,   np.pi*2,   1)

qc.cx(0,1)
qc.cx(2,1)
qc.cx(3,2)

qx = qc.inverse()
print("Inversed circuit using qiskit's method:")
qx.draw(reverse_bits=True)

### 1.2 Quantum State Tomography

1. Implementing circuit in Qiskit

In [ ]:
qc = QuantumCircuit(1)
qc.x(0)
qc.h(0)
qc.tdg(0)
qc.z(0)
qc.y(0)
qc.rx(theta=1.6,qubit=0)
qc.s(0)
print('Circuit:')
display(qc.draw(reverse_bits=True))

2. Retrieve statevector

In [ ]:
statevector = Statevector(qc)
print('Statevector:')
display(statevector.draw('latex'))

3. Apply measurements

In [9]:
# Created function as we will use the same circuit many times

def create_circuit():
    qc = QuantumCircuit(1,1)
    qc.x(0)
    qc.h(0)

    qc.tdg(0)
    qc.z(0)
    qc.y(0)
    qc.rx(theta=1.6,qubit=0)
    qc.s(0)

    return qc

Z-basis (0,1)

In [ ]:
z_basis = create_circuit() # z-basis is the standard basis
z_basis.measure(0,0)
print('Circuit:')
display(z_basis.draw(reverse_bits=True))

statevector = Statevector(qc)
print('Statevector:')
display(statevector.draw('latex'))

X-basis (+,-)

In [ ]:
z_basis = create_circuit() # z-basis is the standard basis

print('Statevector:')
statevector = Statevector(z_basis)
display(statevector.draw('latex'))

print('Output matrix:')
matrix = Operator(z_basis).to_matrix()
display(array_to_latex(matrix))

z_basis.measure(0,0)
print('Circuit:')
display(z_basis.draw(reverse_bits=True))

backend = AerSimulator()
job = backend.run(z_basis, shots=1000)
plot_histogram(job.result().get_counts())

Measuring in the X-basis (+,-)

In [ ]:
x_basis = create_circuit() 

print('Statevector:')
statevector = Statevector(x_basis)
display(statevector.draw('latex'))

x_basis.h(0) # change from standard basis to x-basis
x_basis.measure(0,0)
print('Circuit:')
display(x_basis.draw(reverse_bits=True))

backend = AerSimulator()
job = backend.run(x_basis, shots=1000)
plot_histogram(job.result().get_counts())

Measuring in the Y-basis (R,L)

In [ ]:
y_basis = create_circuit()

print('Statevector:')
statevector = Statevector(y_basis)
display(statevector.draw('latex'))

y_basis.s(0) # # change from standard basis to y-basis
y_basis.measure(0,0)
print('Circuit:')
display(y_basis.draw(reverse_bits=True))

backend = AerSimulator()
job = backend.run(y_basis, shots=1000)
plot_histogram(job.result().get_counts())

4. Reconstruct state from measurments